In [19]:
import andes
import os
import numpy as np
import csv

import matplotlib
%matplotlib inline

# Note

generate unstable data for network training

Example code for batch data generation:

https://docs.andes.app/en/latest/_examples/ex6.html

# Define case path

In [20]:
andes.config_logger(stream_level=50)
# 10-DEBUG，20-INFO30WARNING，40-ERROR，50-CRITICAL

# case path
dir_path = os.path.abspath('..')
loc_path_case = '/Dynamic_123_data_gen_NN_train/case123_REGCV1_v6.xlsx'

case = dir_path + loc_path_case

# Generate settings data

### Input data:

**m_ibr**: virtual inertia for REGCV1 <br> [M1, M2, M3, M4, M5, M6, M7, M8], Range for each M: [0, 6]

**n_ibr**: virtual damping for REGCV1 <br> [D1, D2, D3, D4, D5, D6, D7, D8], Range for each D: [0, 4]

**input** = [m_ibr, n_ibr]

In [21]:
import numpy as np

# Define the number of data points you want
num_samples = 120  # You can change this as needed

# Define the ranges for each variable
m_ibr_range = [0, 1.2]
n_ibr_range = [0, 1]

# Initialize an empty list to store valid data points
input     = []

pg = np.array([0.05 , 0.09 , 0.077, 0.02 , 0.024, 0.014, 0.012, 0.018, 0.015, 0.016, 0.017])

# Generate random data until you have enough valid points
for _ in range(num_samples):
    # Generate random values for m_ibr and n_ibr
    m_ibr = np.random.uniform(m_ibr_range[0], m_ibr_range[1], 8)
    n_ibr = np.random.uniform(n_ibr_range[0], n_ibr_range[1], 8)
    
    # Combine all the data into one array
    combine     = np.concatenate((pg, m_ibr, n_ibr))

    # Append the generated data to the valid_data list
    input.append(combine)


In [22]:
input[0]

array([0.05      , 0.09      , 0.077     , 0.02      , 0.024     ,
       0.014     , 0.012     , 0.018     , 0.015     , 0.016     ,
       0.017     , 0.64883772, 0.03494343, 0.28507674, 0.06292044,
       0.28313438, 0.54020305, 0.05644444, 1.16037947, 0.4225084 ,
       0.23500638, 0.63546926, 0.7941559 , 0.12582797, 0.59832923,
       0.2397145 , 0.1504966 ])

In [23]:
input[0][11:19]

array([0.64883772, 0.03494343, 0.28507674, 0.06292044, 0.28313438,
       0.54020305, 0.05644444, 1.16037947])

In [24]:
input[0][19:27]

array([0.4225084 , 0.23500638, 0.63546926, 0.7941559 , 0.12582797,
       0.59832923, 0.2397145 , 0.1504966 ])

# Batch data generation

### Output data

**rocof_max**: Hz/s, maximum rate of change of frequency

**fnadir** Hz, lowest nadir deviation

**dtheta_max**: rad, maximum angle difference

**eig_max**: maximum eigenvalue before gen trip

**eig_max_**: maximum eigenvalue after gen trip

eig_max = max (eig_max, eig_max_)

**Output** = [rocof_max, fnadir, dtheta_max, eig_max]

In [25]:
output = []

for i in range(num_samples):
    
    # ------------ Reload case -------------
    # --------------------------------------
    ssa = andes.load(case,
                 setup=True,
                 no_output=True)

    # ------- Modify case parameters -------
    # --------------------------------------
    # reset REGCV1 control parameters
    ssa.REGCV1.set(
                    src='M', 
                    idx= ssa.REGCV1.idx.v,
                    attr='v',
                    value= input[i][11:19]
                )
    ssa.REGCV1.set(
                    src='D', 
                    idx= ssa.REGCV1.idx.v, 
                    attr='v',
                    value= input[i][19:27]
                )

    # ------- Run simulation ---------------
    # --------------------------------------
    ssa.PFlow.run()
    ssa.TDS.init()

    # Turn off stability check in TDS
    ssa.TDS.config.criteria = 0
    ssa.TDS.config.no_tqdm = 1    # Turn off simulation progress bar
    ssa.TDS.config.tf = 10        # Simulation time
    ssa.TDS.run()

    # ------- extract data from TDS-----------------
    # --------------------------------------
    # Extract time
    t = ssa.dae.ts.t
    t = t[:, np.newaxis]

    # Extract RoCoF
    rocof     = ssa.dae.ts.y[:,  ssa.BusROCOF.Wf_y.a]
    rocof_max = np.max(np.abs(rocof*60))

    # Extract frequency
    f         = ssa.dae.ts.x[:,  ssa.GENROU.omega.a]
    fnadir    = np.min(f)*60 - 60

    # Extract power angle
    theta      = ssa.dae.ts.y[:, ssa.GENROU.a.a]
    dtheta     = np.max(theta, axis=1) - np.min(theta, axis=1)
    dtheta_max = np.max(dtheta)

    output.append([rocof_max, fnadir, dtheta_max])

<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: 

# Save data

Or save data in the loop

In [26]:
filename = "input_unstable_120.csv"

# Save the list to a CSV file
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(input)

print("Input Data saved to", filename)

Input Data saved to input_unstable_120.csv


In [27]:
filename = "output_unstable_120.csv"

# Save the list to a CSV file
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(output)

print("Output Data saved to", filename)

Output Data saved to output_unstable_120.csv
